In [21]:
import pandas as pd

In [22]:
moldset_labeled = pd.read_csv('./data/labeled_data.csv', index_col=False)

In [23]:
df = pd.read_csv('./data/labeled_data.csv', index_col=False)

In [29]:
df['Reason'].unique()

array([nan, '가스', '미성형', '초기허용불량'], dtype=object)

In [25]:
moldset_labeled.drop(columns=['_id', 'TimeStamp', 'PART_FACT_PLAN_DATE',
                             'PART_FACT_SERIAL', 'EQUIP_CD', 'EQUIP_NAME', 'Reason',
                             'Mold_Temperature_1', 'Mold_Temperature_2','Barrel_Temperature_7','Switch_Over_Position',
                             'Mold_Temperature_5', 'Mold_Temperature_6','Mold_Temperature_7', 'Mold_Temperature_8',
                             'Mold_Temperature_9', 'Mold_Temperature_10', 'Mold_Temperature_11',
                             'Mold_Temperature_12'], inplace=True)

In [26]:
moldset_labeled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7996 entries, 0 to 7995
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   PassOrFail                7996 non-null   object 
 1   Injection_Time            7996 non-null   float64
 2   Filling_Time              7996 non-null   float64
 3   Plasticizing_Time         7996 non-null   float64
 4   Cycle_Time                7996 non-null   float64
 5   Clamp_Close_Time          7996 non-null   float64
 6   Cushion_Position          7996 non-null   float64
 7   Plasticizing_Position     7996 non-null   float64
 8   Clamp_Open_Position       7996 non-null   float64
 9   Max_Injection_Speed       7996 non-null   float64
 10  Max_Screw_RPM             7996 non-null   float64
 11  Average_Screw_RPM         7996 non-null   float64
 12  Max_Injection_Pressure    7996 non-null   float64
 13  Max_Switch_Over_Pressure  7996 non-null   float64
 14  Max_Back

In [27]:
moldset_labeled["PassOrFail"] = moldset_labeled["PassOrFail"].replace(to_replace="Y",value=1).replace(to_replace="N",value=0)

C:\Users\Admin\AppData\Local\Temp\ipykernel_9320\2662115890.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  moldset_labeled["PassOrFail"] = moldset_labeled["PassOrFail"].replace(to_replace="Y",value=1).replace(to_replace="N",value=0)


In [28]:
moldset_labeled["PART_NAME"].value_counts()

KeyError: 'PART_NAME'

In [ ]:

rg3_r = moldset_labeled[moldset_labeled["PART_NAME"] == "RG3 MOLD'G W/SHLD, RH"].copy()
rg3_l = moldset_labeled[moldset_labeled["PART_NAME"] == "RG3 MOLD'G W/SHLD, LH"].copy()
rg3_r.drop(columns=['PART_NAME'], inplace=True)
rg3_l.drop(columns=['PART_NAME'], inplace=True)

In [ ]:

cn7_r = moldset_labeled[moldset_labeled["PART_NAME"] == "CN7 W/S SIDE MLD'G RH"].copy()
cn7_l = moldset_labeled[moldset_labeled["PART_NAME"] == "CN7 W/S SIDE MLD'G LH"].copy()
cn7_r.drop(columns=['PART_NAME'], inplace=True)
cn7_l.drop(columns=['PART_NAME'], inplace=True)

In [ ]:
y = cn7_r['PassOrFail'].copy()
X = cn7_r.drop(columns=['PassOrFail']).copy()
y_test = rg3_l['PassOrFail'].copy()
X_test = rg3_l.drop(columns=['PassOrFail']).copy()
X.shape, y.shape, X_test.shape, y_test.shape

In [10]:
from sklearn.model_selection import train_test_split, GridSearchCV

In [11]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score

In [12]:
ss = StandardScaler()
ss.fit(X)
X = ss.transform(X)
X_test = ss.transform(X_test)

In [13]:
# 모델 생성
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, HistGradientBoostingClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier

rf = RandomForestClassifier(random_state=42)
et = ExtraTreesClassifier(random_state=42)
gb = GradientBoostingClassifier(random_state=42)
hgb = HistGradientBoostingClassifier(random_state=42)
xgb = XGBClassifier(random_state=42)

model_list = [rf, et, gb, hgb, xgb]

In [14]:
# grid_df = pd.DataFrame()
# scores = ['accuracy', 'precision', 'recall', 'f1']
# for score in scores:
#     # GridSearchCV 이용해서 모든 모델 하이퍼파라미터 튜닝
#     for model in model_list:
#         model_name = model.__class__.__name__
#         gridParams = dict()
#         if model_name =="XGBClassifier" :
#             gridParams["n_estimators"] = [20,50]
#             gridParams["max_depth"] = [10,20]
#             gridParams["min_child_weight"] = [1, 2,4]
#         elif model_name == "HistGradientBoostingClassifier" :
#             gridParams["max_iter"] = [20, 50]
#             gridParams["max_depth"] = [10, 50]
#             gridParams["min_samples_leaf"] = [1,2,4]
#         else :
#             gridParams["n_estimators"] = [20, 50]
#             gridParams["max_depth"] = [10,20]
#             gridParams["min_samples_split"] = [2,5,10]
#             gridParams["min_samples_leaf"] = [1,2,4]

#         grid_search_model = GridSearchCV(model, gridParams, scoring=score, cv=5, n_jobs=-1)
#         grid_search_model.fit(X, y)
#         model = grid_search_model.best_estimator_
#         pred = model.predict(X)
#         acc = accuracy_score(y, pred)
#         pre = precision_score(y, pred)
#         rec = recall_score(y, pred)
#         f1 = f1_score(y, pred)
#         df_temp = pd.DataFrame([[model_name,  acc, pre, rec, f1, str(grid_search_model.best_params_)]],
#                                  columns=["모델명", "정확도", 
#                                           "정밀도", "재현율", "f1-score", "파라미터"])
#         grid_df = pd.concat([grid_df, df_temp], ignore_index=True)

In [15]:
# grid_df

In [16]:
gridParams = dict()
gridParams["n_estimators"] = [20, 50]
gridParams["max_depth"] = [10,20]
gridParams["min_samples_split"] = [2,5,10]
gridParams["min_samples_leaf"] = [1,2,4]
grid_search_model = GridSearchCV(RandomForestClassifier(), gridParams, cv=5, n_jobs=-1)
grid_search_model.fit(X, y)
model = grid_search_model.best_estimator_

In [17]:
pred = model.predict(X_test)
acc = accuracy_score(y_test, pred)
pre = precision_score(y_test, pred)
rec = recall_score(y_test, pred)
f1 = f1_score(y_test, pred)
acc, pre, rec, f1

(0.9920382165605095, 0.9920382165605095, 1.0, 0.9960031974420464)